### Needed to install iisignature

In [69]:
!apt install gcc g++ python3-dev build-essential -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
g++ is already the newest version (4:11.2.0-1ubuntu1).
gcc is already the newest version (4:11.2.0-1ubuntu1).
python3-dev is already the newest version (3.10.6-1~22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [70]:
!pip install "jax[cuda]" tensorflow torch keras jaxtyping signax iisignature iisignature_tensorflow_2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 81.2 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2


In [1]:
!pip install keras jaxtyping signax iisignature iisignature_tensorflow_2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.3/87.3 MB 13.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 45.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 68.5 MB/s eta 0:00:00a 0:00:01
  Created wheel for iisignature: filename=iisignature-0.24-cp311-cp311-linux_x86_64.whl size=3246131 sha256=6dff37c3f540590b75d983d00d51e282322fb003d6210375fd2af5f494129256
  Stored in directory: /root/.cache/pip/wheels/1c/f4/57/0b4d3787a07f20a3cd1a91835d6247f55ef899345267bcd6df
Successfully built iisignature
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, 

In [2]:
# Standard library imports
import os
import time
from functools import partial
from typing import Callable, List, Optional, Union

# Third-party scientific computing imports
import numpy as np
import iisignature
import signax

# Keras and backend configuration
BACKEND = 'tensorflow'
os.environ['KERAS_BACKEND'] = BACKEND

import keras
from keras import ops

# Backend-specific type definitions
backend = keras.backend.backend()
if backend == 'tensorflow':
    import tensorflow as tf
    Array = tf.Tensor
elif backend == 'jax':
    from jaxtyping import Array, Float
elif backend == 'torch':
    import torch
    Array = torch.Tensor

2024-11-21 18:31:12.144906: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732213872.161853     638 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732213872.166912     638 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Matrix Operations

In [49]:
def otimes(x: Array, y: Array) -> Array:
    """Tensor product

    Args:
        x: size=(n,n,...,n), ndim=ndim_x
        y: size=(n,n,...,n), ndim=ndim_y
    Return:
        Tensor size (n,n,...,n) with ndim=ndim_x + ndim_y
    """
    expanded_x = ops.reshape(x, x.shape + (1,) * y.ndim)
    expanded_y = ops.reshape(y, (1,) * x.ndim + y.shape)
    return expanded_x * expanded_y


def addcmul(x: Array, y: Array, z: Array) -> Array:
    """Similar to `torch.addcmul` returning
        x + y * z
    Here `*` is the tensor product
    """
    return x + otimes(y, z)


def restricted_exp(input: Array, depth: int) -> list[Array]:
    """Restricted exponentiate

    As `depth` is fixed so we can make it as a static argument.
    This allows us to `jit` this function
    Args:
        input: shape (n, )
        depth: the depth of signature
    Return:
        A list of `jnp.ndarray` contains tensors
    """
    ret = [input]
    for i in range(2, depth + 1):
        ret.append(otimes(ret[-1], input / i))
    return ret


def mult_fused_restricted_exp(z: Array, A: list[Array]) -> list[Array]:
    """
    Multiply-fused-exponentiate

    Args:
        z: shape (n,)
        A: a list of `jnp.array` [(n, ), (n x n), (n x n x n), ...]
    Return:
        A list of which elements have the same shape with `A`
    """
    depth = len(A)
    ret = []

    for depth_index in range(depth):
        current = ops.array(1.0, dtype=z.dtype)
        for i in range(depth_index + 1):
            current = addcmul(x=A[i], y=current, z=z / (depth_index + 1 - i))
        ret.append(current)

    return ret

def batch_otimes(x: Array, y: Array) -> Array:
    """Tensor product

    Args:
        x: size=(n,n,...,n), ndim=ndim_x
        y: size=(n,n,...,n), ndim=ndim_y
    Return:
        Tensor size (n,n,...,n) with ndim=ndim_x + ndim_y
    """
    expanded_x = ops.reshape(x, (x.shape + (1,) * (y.ndim - 1)))
    expanded_y = ops.reshape(y, (y.shape[0],) + (1,) * (x.ndim - 1) + y.shape[1:])
    return expanded_x * expanded_y
    
def batch_restricted_exp(input: Array, depth: int) -> list[Array]:
    """Restricted exponentiate

    As `depth` is fixed so we can make it as a static argument.
    This allows us to `jit` this function
    Args:
        input: shape (n, )
        depth: the depth of signature
    Return:
        A list of `jnp.ndarray` contains tensors
    """
    ret = [input]
    for i in range(2, depth + 1):
        ret.append(batch_otimes(ret[-1], input / i))
    return ret

def batch_mult_fused_restricted_exp(z: Array, A: list[Array]) -> list[Array]:
    """
    Multiply-fused-exponentiate

    Args:
        z: shape (n,)
        A: a list of `jnp.array` [(n, ), (n x n), (n x n x n), ...]
    Return:
        A list of which elements have the same shape with `A`
    """
    depth = len(A)
    ret = []

    for depth_index in range(depth):
        current = ops.array(1.0, dtype=z.dtype)
        for i in range(depth_index + 1):
            current = batch_addcmul(x=A[i], y=current, z=z / (depth_index + 1 - i))
        ret.append(current)

    return ret

def batch_addcmul(x: Array, y: Array, z: Array) -> Array:
    """Similar to `torch.addcmul` returning
        x + y * z
    Here `*` is the tensor product
    """
    return x + batch_otimes(y, z)


# Signature computation

In [45]:
def get_largest_divisor_under_20(n: int) -> int:
    """Get largest divisor under 20 for scan unrolling"""
    for i in range(20, 0, -1):
        if i <= n and n % i == 0:
            return i
    return 10

def signature(
    path: Array,
    depth: int,
    stream: bool = False,
    unroll:Optional[Union[bool, int]] = None,
) -> list[Array] | Array:
    """
    Compute the signature of a path. Automatically dispatches to vmap or not based on the shape of `path`.

    Args:
        path: size (length, dim) or (batch, length, dim)
        depth: signature is truncated at this depth
        stream: whether to handle `path` as a stream. Default is False
        flatten: whether to flatten the output. Default is False
        num_chunks: number of chunks to use. Default is 1. If > 1, path will be divided into
        chunks to compute signatures. Then, obtained signatures are combined (using Chen's identity).

    Returns:
        If `stream` is `True`, this will return a list of `Array` in a form
            [(path_len - 1, dim), (path_len - 1, dim, dim), (path_len - 1, dim, dim, dim), ...]
        If `stream` is `False`, this will return a list of `Array` in a form
            [(dim, ), (dim, dim), (dim, dim, dim), ...]
        If `flatten` is `True`, this will return a flattened array of shape
            (dim + dim**2 + ... + dim**depth, )
        If your path is of shape (batch, path_len, dim), all of the above will have an extra
        dimension of size `batch` as the first dimension.

    """

    # this is just to handle shape errors
    if path.ndim == 2:
        return _single_signature(path, depth=depth, stream=stream, unroll=unroll)
    if path.ndim == 3:  # batch case (mimics signatory)
        return _batch_signature(path, depth=depth, stream=stream, unroll=unroll)
    msg = f"Path must be of shape (path_length, path_dim) or (batch, path_length, path_dim), got {path.shape}"
    raise ValueError(msg)

def _single_signature(
    path: Array,
    depth: int,
    stream: bool = False,
    unroll: Optional[Union[bool, int]] = False,
) -> list[Array] | Array:
    """
    Compute the signature of a path. Optionally, divide the path into chunks to compute signatures
    and combine them using Chen's identity (useful for long paths).

    Args:
        path: size (length, dim)
        depth: signature is truncated at this depth
        stream: whether to handle `path` as a stream. Default is False

        If `stream` is `True`, this will return an array of form
            (path_len - 1, sum(dim**i for i in range depth))
        If `stream` is `False`, this will return an array in a form
            (sum(dim**i for i in range depth))
    """
    path_increments = path[1:,:] - path[:-1,:]
    exp_term = restricted_exp(path_increments[0], depth=depth)

    carry = exp_term
    if stream:
        stacked =[carry]
    for inc_idx in range(1, path_increments.shape[0]):
        inc = path_increments[inc_idx]
        carry = mult_fused_restricted_exp(inc, carry)
        if stream:
            stacked.append(carry)
        
    if stream:
        res = ops.stack([
            ops.concatenate([ops.reshape(c, (-1,)) for c in res])
            for res in stacked
        ], axis=0)

    else:
        res = carry
        # `res` has shape [(dim,), (dim, dim), ...]
        res = ops.concatenate([ops.reshape(c, (-1,)) for c in res])
           
    return res


def _batch_signature(
    path: Array,
    depth: int,
    stream: bool = False,
    unroll: Optional[Union[bool, int]] = None,
) -> List[Array] | Array:
    """
    Compute the signature of a path matching original implementation exactly.
    """
    batch_size = path.shape[0]
    seq_len = path.shape[1]
    path_increments = path[:, 1:] - path[:, :-1]
    exp_term = batch_restricted_exp(path_increments[:, 0], depth=depth)
    unroll_level = unroll if unroll is not None else get_largest_divisor_under_20(seq_len-1)
    
    if not stream:
        # Non-streaming case
        def scan_fn(carry, increment):
            new_carry = batch_mult_fused_restricted_exp(increment, carry)
            return new_carry, None
            
        final_carry, _ = ops.scan(
            scan_fn,
            init=exp_term,
            xs=ops.moveaxis(path_increments[:, 1:], 1, 0),  # Reshape to (steps, batch, dim)
            unroll=unroll_level,
            length=path_increments.shape[1] - 1
        )
        # Match original format: concatenate at the very end
        return ops.concatenate([ops.reshape(c, (batch_size,-1)) for c in final_carry], axis=1)
    else:
        # Streaming case
        def scan_fn(carry, increment):
            new_carry = batch_mult_fused_restricted_exp(increment, carry)
            return new_carry, new_carry
        
        _, stacked = ops.scan(
            scan_fn,
            init=exp_term,
            xs=ops.moveaxis(path_increments[:, 1:], 1, 0),  # Shape: (batch, steps, dim)
            unroll=unroll_level,
            length=path_increments.shape[1] - 1
        )
        res = [
            ops.concatenate([first[None, ...], rest], axis=0)
            for first, rest in zip(exp_term, stacked)
        ]
        # Match original stacking and concatenation
        return ops.concatenate([ops.reshape(ops.moveaxis(r, 1, 0), (batch_size, seq_len-1, -1)) for r in res], axis=2)

#Needed a tensorflow variant without scan 

def tf_signature(
    path,
    depth: int,
    stream: bool = False,
    unroll: int = 1,
) -> list[Array] | Array:
    """
    Compute the signature of a path. Automatically dispatches to vmap or not based on the shape of `path`.

    Args:
        path: size (length, dim) or (batch, length, dim)
        depth: signature is truncated at this depth
        stream: whether to handle `path` as a stream. Default is False
        flatten: whether to flatten the output. Default is False
        num_chunks: number of chunks to use. Default is 1. If > 1, path will be divided into
        chunks to compute signatures. Then, obtained signatures are combined (using Chen's identity).

    Returns:
        If `stream` is `True`, this will return a list of `Array` in a form
            [(path_len - 1, dim), (path_len - 1, dim, dim), (path_len - 1, dim, dim, dim), ...]
        If `stream` is `False`, this will return a list of `Array` in a form
            [(dim, ), (dim, dim), (dim, dim, dim), ...]
        If `flatten` is `True`, this will return a flattened array of shape
            (dim + dim**2 + ... + dim**depth, )
        If your path is of shape (batch, path_len, dim), all of the above will have an extra
        dimension of size `batch` as the first dimension.

    """

    # this is just to handle shape errors
    if path.ndim == 2:
        return _tf_single_signature(path, depth=depth, stream=stream)
    if path.ndim == 3:  # batch case (mimics signatory)
        return _tf_batch_signature(path, depth=depth, stream=stream)
    msg = f"Path must be of shape (path_length, path_dim) or (batch, path_length, path_dim), got {path.shape}"
    raise ValueError(msg)

def _tf_single_signature(
    path,
    depth: int,
    stream: bool = False,
) -> list[Array] | Array:
    """
    Compute the signature of a path. Optionally, divide the path into chunks to compute signatures
    and combine them using Chen's identity (useful for long paths).

    Args:
        path: size (length, dim)
        depth: signature is truncated at this depth
        stream: whether to handle `path` as a stream. Default is False

        If `stream` is `True`, this will return an array of form
            (path_len - 1, sum(dim**i for i in range depth))
        If `stream` is `False`, this will return an array in a form
            (sum(dim**i for i in range depth))
    """
    path_increments = path[1:,:] - path[:-1,:]
    exp_term = restricted_exp(path_increments[0], depth=depth)

    carry = exp_term
    if stream:
        stacked =[carry]
    for inc_idx in range(1, path_increments.shape[0]):
        inc = path_increments[inc_idx]
        carry = mult_fused_restricted_exp(inc, carry)
        if stream:
            stacked.append(carry)
        
    if stream:
        res = ops.stack([
            ops.concatenate([ops.reshape(c, (-1,)) for c in res])
            for res in stacked
        ], axis=0)

    else:
        res = carry
        # `res` has shape [(dim,), (dim, dim), ...]
        res = ops.concatenate([ops.reshape(c, (-1,)) for c in res])
           
    return res

def _tf_batch_signature(
    path,
    depth: int,
    stream: bool = False,
) -> list[Array] | Array:
    """
    Compute the signature of a path. Optionally, divide the path into chunks to compute signatures
    and combine them using Chen's identity (useful for long paths).

    Args:
        path: size (length, dim)
        depth: signature is truncated at this depth
        stream: whether to handle `path` as a stream. Default is False

        If `stream` is `True`, this will return an array of form
            (path_len - 1, sum(dim**i for i in range depth))
        If `stream` is `False`, this will return an array in a form
            (sum(dim**i for i in range depth))
    """
    batch_size = path.shape[0]
    path_increments = path[:,1:] - path[:,:-1]
    exp_term = batch_restricted_exp(path_increments[:, 0], depth=depth)
    
    carry = exp_term
    if stream:
        stacked =[carry]
    for inc_idx in range(1, path_increments.shape[1]):
        inc = path_increments[:, inc_idx]
        carry = batch_mult_fused_restricted_exp(inc, carry)   
        if stream:
            stacked.append(carry)
    if stream:
        res = ops.stack([
            ops.concatenate([ops.reshape(c, (batch_size,-1,)) for c in res], axis=1)
            for res in stacked
        ], axis=1)
    else:
        res = carry
        res = ops.concatenate([ops.reshape(c, (batch_size,-1,)) for c in res], axis=1)
    return res



# Pure signature computation performance versus iisignature and signax

## Runed with Jax Backend for Keras

In [98]:
paths = np.random.randn(10000,100,5)
depth = 4

iisig = iisignature.sig(paths, depth)
signaxsig = signax.signature(paths, depth)
kerassig = signature(paths, depth)

assert np.max(np.abs(iisig - signaxsig)) < 1e-3 and np.max(np.abs(iisig - kerassig)) < 1e-3

%timeit iisignature.sig(paths, depth)
%timeit signax.signature(paths, depth)
%timeit signature(paths, depth)

2.86 s ± 9.62 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
21 ms ± 4.66 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
758 ms ± 18.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Conclusion: 

For single forward pace signax is the most optimal (unsurprisingly), as it avoid many inderection and can make real use of jax functions directly

Still our proposed approach still overperform the standard iisignature one, but this is only true with high batch size / signature level / number of features

## Runed with Tensorflow Backend for Keras

In [5]:
paths = np.random.randn(1000,100,5)
depth = 4

iisig = iisignature.sig(paths, depth)
signaxsig = signax.signature(paths, depth)
kerassig = signature(paths, depth)

assert np.max(np.abs(iisig - signaxsig)) < 1e-3 and np.max(np.abs(iisig - kerassig)) < 1e-3

%timeit iisignature.sig(paths, depth)
%timeit signax.signature(paths, depth)
%timeit signature(paths, depth)

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.
W0000 00:00:1732213874.612505     638 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


237 ms ± 433 μs per loop (mean ± std. dev. of 7 runs, 1 loop each)
43.3 ms ± 1.19 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
958 ms ± 37.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Conclusion

With a number of element divided by 10 the time taken is still the same for the keras sig as with the jax backend.

However it's interesting to note that this run has been done on a machine with a non cuda version of jax, resulting in signax performance degraded in comparison. It thus seems to appears that our proposed method in forward pass difference of performance lies in the use of the GPU by jax directly while keras only used it on compiled model at train time.

## Runed with Torch Backend

In [11]:
paths = np.random.randn(1000,100,5)
depth = 4

iisig = iisignature.sig(paths, depth)
signaxsig = signax.signature(paths, depth)
kerassig = signature(paths, depth)

assert np.max(np.abs(iisig - signaxsig)) < 1e-3 and np.max(np.abs(iisig - kerassig.cpu().numpy())) < 1e-3

%timeit iisignature.sig(paths, depth)
%timeit signax.signature(paths, depth)
%timeit signature(paths, depth)

284 ms ± 1.97 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.43 ms ± 786 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
143 ms ± 860 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Conclusion

Torch is really better than tensorflow here for forward pass, but not at jax level

# Signature Layer

In [15]:
from iisignature_tensorflow_2 import SigLayer as iisigSigLayer

class SigLayer(keras.layers.Layer):
    def __init__(self, depth, stream = False, unroll = None, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.depth = depth
        self.stream = stream
        self.unroll = unroll
        self.signature_func = signature if keras.backend.backend()!="tensorflow" else tf_signature
        
    def call(self, inputs):
        return self.signature_func(inputs, depth=self.depth, stream=self.stream, unroll=self.unroll)


class SignaxSigLayer(keras.layers.Layer):
    def __init__(self, depth, stream = False, unroll = None, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.depth = depth
        self.stream = stream
        
    def call(self, inputs):
        return signax.signature(inputs, depth=self.depth, stream=self.stream)

# Example in SigDense case

In [10]:
from keras.layers import Layer, Dense, Add, LayerNormalization, Multiply, Dropout
from keras.layers import Input, Flatten

class AddAndNorm(Layer):
    def __init__(self, **kwargs):
        super(AddAndNorm, self).__init__(**kwargs)
        self.add_layer = Add()
        self.norm_layer = LayerNormalization()
    
    def call(self, inputs):
        tmp = self.add_layer(inputs)
        tmp = self.norm_layer(tmp)
        return tmp

    def compute_output_shape(self, input_shape):
        return input_shape[0]  # Assuming all input shapes are the same

class Gate(Layer):
    def __init__(self, hidden_layer_size = None, **kwargs):
        super(Gate, self).__init__(**kwargs)
        self.hidden_layer_size = hidden_layer_size

    def build(self, input_shape):
        if self.hidden_layer_size is None:
            self.hidden_layer_size = input_shape[-1]
        self.dense_layer = Dense(self.hidden_layer_size)
        self.gated_layer = Dense(self.hidden_layer_size, activation='sigmoid')
        super(Gate, self).build(input_shape)

    def call(self, inputs):
        dense_output = self.dense_layer(inputs)
        gated_output = self.gated_layer(inputs)
        return Multiply()([dense_output, gated_output])

    def compute_output_shape(self, input_shape):
        return input_shape[:-1] + (self.hidden_layer_size,)

class GRN(Layer):
    def __init__(self, hidden_layer_size, output_size=None, activation = None, dropout = 0.1, **kwargs):
        super(GRN, self).__init__(**kwargs)
        self.hidden_layer_size = hidden_layer_size
        self.output_size = output_size
        self.activation = keras.activations.get(activation) if activation is not None else None
        self.dropout = Dropout(dropout)
        self.dropout_value = dropout

    def build(self, input_shape):
        if self.output_size is None:
            self.output_size = self.hidden_layer_size
        self.skip_layer = Dense(self.output_size)
        
        self.hidden_layer_1 = Dense(self.hidden_layer_size, activation='elu')
        self.hidden_layer_2 = Dense(self.hidden_layer_size)
        self.gate_layer = Gate(self.output_size)
        self.add_and_norm_layer = AddAndNorm()
        super(GRN, self).build(input_shape)

    def call(self, inputs):
        if self.skip_layer is None:
            skip = inputs
        else:
            skip = self.skip_layer(inputs)
        
        hidden = self.hidden_layer_1(inputs)
        hidden = self.hidden_layer_2(hidden)
        hidden = self.dropout(hidden)
        gating_output = self.gate_layer(hidden)
        output = self.add_and_norm_layer([skip, gating_output])
        if self.activation is not None:
            output = self.activation(output)
        return output

    def compute_output_shape(self, input_shape):
        return input_shape[:-1] + (self.output_size,)

class SigDense(Layer):
    def __init__(self, unit, sig_level, siglayer_class, dropout = 0., **kwargs):
        super().__init__(**kwargs)
        self.unit = unit
        self.sig_level = sig_level
        self.sig_layer = siglayer_class(self.sig_level)
        self.dense_layer = Dense(unit)
        self.sig_to_weight = GRN(unit, activation = 'softmax', dropout = dropout)
        self.dropout = Dropout(dropout)

    def build(self, input_shape):
        _, seq_length, n_features = input_shape
        name = self.name
        self.time_weigthing_kernel = self.add_weight(
            shape=(seq_length, 1),
            name=f"{name}_time_weigthing_kernel",
        )
        super().build(input_shape)
        
    def call(self, inputs):
        inputs = self.time_weigthing_kernel * inputs
        sig = self.sig_layer(inputs)
        weights = self.sig_to_weight(sig)
        dense_out = self.dense_layer(inputs)
        dense_out = self.dropout(dense_out)
        weights = keras.ops.expand_dims(weights, axis=1)
        return dense_out * weights

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[1], self.unit)


## Runed with Jax Backend for Keras

In [113]:
BATCH_SIZE = 128
num_sample = BATCH_SIZE*100 - 35  #In order to have an not exactly equal to Batch size total number of sample 
seq_len = 500
n_feature = 10
n_ahead = 10
depth = 3
X = np.random.randn(num_sample, seq_len, n_feature)
y = np.random.randn(num_sample, n_ahead)

t1 = time.time()
model = keras.Sequential([
    Input(shape=X.shape[1:]),
    Dense(7),
    SigDense(10, depth, SigLayer),
    Flatten(),
    Dense(10, 'relu'),
    Dense(n_ahead),
])
model.summary()
model.compile('adam', 'mse', jit_compile=True)
model.fit(X, y, batch_size=BATCH_SIZE, epochs=10, verbose=True)
print("Time with Pure Keras Signature", time.time() - t1)
del model

t1 = time.time()
model = keras.Sequential([
    Input(shape=X.shape[1:]),
    Dense(6),
    SigDense(10, depth, SignaxSigLayer),
    Flatten(),
    Dense(10, 'relu'),
    Dense(n_ahead),
])
model.summary()
model.compile('adam', 'mse', jit_compile=True)
model.fit(X, y, batch_size=BATCH_SIZE, epochs=10, verbose=True)
print("Time with Pure Jax Signature from Signax", time.time() - t1)
del model

Model: "sequential_21"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_169 (Dense)               │ (None, 500, 7)         │            77 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sig_dense_21 (SigDense)         │ (None, 500, 10)        │           500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_21 (Flatten)            │ (None, 5000)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_171 (Dense)               │ (None, 10)             │        50,010 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_172 (Dense)               │ (None, 10)             │           110 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 50,697 (198.04 KB)

 Trainable params: 50,697 (198.04 KB)

 Non-trainable params: 0 (0.00 B)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_178 (Dense)               │ (None, 500, 6)         │            66 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sig_dense_22 (SigDense)         │ (None, 500, 10)        │           500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_22 (Flatten)            │ (None, 5000)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_180 (Dense)               │ (None, 10)             │        50,010 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_181 (Dense)               │ (None, 10)             │           110 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 50,686 (197.99 KB)

 Trainable params: 50,686 (197.99 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 7s 48ms/step - loss: 0.9997
Epoch 2/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - loss: 0.9929
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.9720
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.9189
Epoch 5/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.8615
Epoch 6/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.8110
Epoch 7/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.7772
Epoch 8/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.7374
Epoch 9/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.7186
Epoch 10/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.6977
Time with Pure Jax Signature from Signax 21.143450260162354


## Conclusion:
While signax signature is more efficient alone for forward computation, inside keras backend the direct use is not distinguishable from our proposed keras signature 

iisignature_tensorflow_2 is obviously uncompatible

## Runed with Tensorflow Backend 

In [51]:
BATCH_SIZE = 128
num_sample = BATCH_SIZE*100 #- 35  #In order to have an not exactly equal to Batch size total number of sample 
seq_len = 50
n_feature = 10
n_ahead = 10
depth = 3
X = np.random.randn(num_sample, seq_len, n_feature)
y = np.random.randn(num_sample, n_ahead)

t1 = time.time()
model = keras.Sequential([
    Input(shape=X.shape[1:]),
    Dense(7),
    SigDense(10, depth, SigLayer),
    Flatten(),
    Dense(10, 'relu'),
    Dense(n_ahead),
])
model.summary()
model.compile('adam', 'mse', jit_compile=True)
model.fit(X, y, batch_size=BATCH_SIZE, epochs=10, verbose=True)
print("Time with Pure Keras Signature", time.time() - t1)
del model

t1 = time.time()
model = keras.Sequential([
    Input(shape=X.shape[1:]),
    Dense(6),
    SigDense(10, depth, iisigSigLayer),
    Flatten(),
    Dense(10, 'relu'),
    Dense(n_ahead),
])
model.summary()
model.compile('adam', 'mse', jit_compile=False)
model.fit(X, y, batch_size=BATCH_SIZE, epochs=10, verbose=True)
print("Time with iisignature Wrapper in tensorflow", time.time() - t1)
del model

Model: "sequential_20"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_136 (Dense)               │ (None, 50, 7)          │            77 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sig_dense_20 (SigDense)         │ (None, 50, 10)         │            50 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_20 (Flatten)            │ (None, 500)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_138 (Dense)               │ (None, 10)             │         5,010 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_139 (Dense)               │ (None, 10)             │           110 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,247 (20.50 KB)

 Trainable params: 5,247 (20.50 KB)

 Non-trainable params: 0 (0.00 B)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_145 (Dense)               │ (None, 50, 6)          │            66 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sig_dense_21 (SigDense)         │ (None, 50, 10)         │            50 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_21 (Flatten)            │ (None, 500)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_147 (Dense)               │ (None, 10)             │         5,010 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_148 (Dense)               │ (None, 10)             │           110 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,236 (20.45 KB)

 Trainable params: 5,236 (20.45 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 7s 49ms/step - loss: 0.9985
Epoch 2/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 5s 47ms/step - loss: 0.9963
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 4s 43ms/step - loss: 1.0016
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 4s 44ms/step - loss: 0.9967
Epoch 5/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 4s 42ms/step - loss: 0.9852
Epoch 6/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 5s 46ms/step - loss: 0.9881
Epoch 7/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 5s 48ms/step - loss: 0.9778
Epoch 8/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 5s 45ms/step - loss: 0.9640
Epoch 9/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 5s 46ms/step - loss: 0.9667
Epoch 10/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 5s 46ms/step - loss: 0.9610
Time with iisignature Wrapper in tensorflow 48.41623115539551


In [52]:
BATCH_SIZE = 128
num_sample = BATCH_SIZE*100 #- 35  #In order to have an not exactly equal to Batch size total number of sample 
seq_len = 20
n_feature = 10
n_ahead = 10
depth = 4
X = np.random.randn(num_sample, seq_len, n_feature)
y = np.random.randn(num_sample, n_ahead)

t1 = time.time()
model = keras.Sequential([
    Input(shape=X.shape[1:]),
    Dense(7),
    SigDense(10, depth, SigLayer),
    Flatten(),
    Dense(10, 'relu'),
    Dense(n_ahead),
])
model.summary()
model.compile('adam', 'mse', jit_compile=True)
model.fit(X, y, batch_size=BATCH_SIZE, epochs=10, verbose=True)
print("Time with Pure Keras Signature", time.time() - t1)
del model

t1 = time.time()
model = keras.Sequential([
    Input(shape=X.shape[1:]),
    Dense(6),
    SigDense(10, depth, iisigSigLayer),
    Flatten(),
    Dense(10, 'relu'),
    Dense(n_ahead),
])
model.summary()
model.compile('adam', 'mse', jit_compile=False)
model.fit(X, y, batch_size=BATCH_SIZE, epochs=10, verbose=True)
print("Time with iisignature Wrapper in tensorflow", time.time() - t1)
del model

Model: "sequential_22"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_154 (Dense)               │ (None, 20, 7)          │            77 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sig_dense_22 (SigDense)         │ (None, 20, 10)         │            20 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_22 (Flatten)            │ (None, 200)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_156 (Dense)               │ (None, 10)             │         2,010 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_157 (Dense)               │ (None, 10)             │           110 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,217 (8.66 KB)

 Trainable params: 2,217 (8.66 KB)

 Non-trainable params: 0 (0.00 B)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_163 (Dense)               │ (None, 20, 6)          │            66 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sig_dense_23 (SigDense)         │ (None, 20, 10)         │            20 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_23 (Flatten)            │ (None, 200)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_165 (Dense)               │ (None, 10)             │         2,010 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_166 (Dense)               │ (None, 10)             │           110 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,206 (8.62 KB)

 Trainable params: 2,206 (8.62 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 12s 97ms/step - loss: 0.9898
Epoch 2/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 9s 93ms/step - loss: 0.9863
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 9s 89ms/step - loss: 0.9922
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 10s 95ms/step - loss: 0.9812
Epoch 5/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 9s 94ms/step - loss: 0.9751
Epoch 6/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 9s 91ms/step - loss: 0.9588
Epoch 7/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 9s 91ms/step - loss: 0.9402
Epoch 8/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 9s 95ms/step - loss: 0.9382
Epoch 9/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 9s 91ms/step - loss: 0.9304
Epoch 10/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 9s 92ms/step - loss: 0.9197
Time with iisignature Wrapper in tensorflow 95.6760663986206


## Conclusion:

Model is compilable in tensorflow with our proposed layer which is not the case when using the iisignature wrapper, thus enhancing the performance of training. 

Still, the performance are still not reaching jax ones, and something to point out is the fact we can't use scan to limit unrolling of loops, creating huge up-front compile time, thus reducing the maximum sequence length we could use.

It's ideal for short sequences with high signature degree but not for long sequence.

# Runned with Torch Backend

In [7]:
BATCH_SIZE = 128
num_sample = BATCH_SIZE*100 - 35  #In order to have an not exactly equal to Batch size total number of sample 
seq_len = 500
n_feature = 10
n_ahead = 10
depth = 3
X = np.random.randn(num_sample, seq_len, n_feature)
y = np.random.randn(num_sample, n_ahead)

t1 = time.time()
model = keras.Sequential([
    Input(shape=X.shape[1:]),
    Dense(7),
    SigDense(10, depth, SigLayer),
    Flatten(),
    Dense(10, 'relu'),
    Dense(n_ahead),
])
model.summary()
model.compile('adam', 'mse', jit_compile=False)
model.fit(X, y, batch_size=BATCH_SIZE, epochs=2, verbose=True)
print("Time with Pure Keras Signature non compiled using SigDense", time.time() - t1)
del model


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 500, 7)         │            77 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sig_dense (SigDense)            │ (None, 500, 10)        │           500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 5000)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 10)             │        50,010 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 10)             │           110 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 50,697 (198.04 KB)

 Trainable params: 50,697 (198.04 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/2
100/100 ━━━━━━━━━━━━━━━━━━━━ 93s 927ms/step - loss: 1.0036
Epoch 2/2
100/100 ━━━━━━━━━━━━━━━━━━━━ 92s 922ms/step - loss: 0.9967
Time with Pure Keras Signature non compiled using SigDense 187.0139937400818


## Conclusion:

Working but far from suboptimal - model compilation break due to the gate layer, but we also tried in another simpler architecture it break too.
This seems to come from torch dynamo compiler.
Even if working, we would not recommend using it.